# Application: Creating a digital twin
This notebook contains code to create an agent and prompt it to critique some content. Edit the agent traits and survey details to have a different agent review something else, and add/edit questions and language models as you see fit. 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/expectedparrot/edsl/blob/main/notebooks/digital_twin.ipynb)

In [1]:
# ! pip install edsl

In [2]:
from edsl.questions import QuestionMultipleChoice, QuestionCheckBox, QuestionFreeText, QuestionLinearScale, QuestionList, QuestionBudget
from edsl import Agent, Scenario, Survey, Model

In [3]:
# Construct relevant traits as a dictionary
agent_traits = {
    "persona": """You are a middle-aged mom in Cambridge, Massachusetts. You hope to own a driverless minivan in the near future.
        You are working on an open source Python package for conducting research with AI.""",
    "age": 44,
    "location": "US",
    "industry": "information technology",
    "company": "Expected Parrot",
    "occupation": "startup cofounder",
    "hobbies": "kayaking, beach walks"
}

# Pass the traits and an optional name to an agent
agent = Agent(name = "Robin", traits = agent_traits)

# Optionally create some special instructions for the task
context = """You are answering questions about a software package for conducting surveys and experiments 
          with large language models. The creators of the software want to know your opinions about some
          new features they are considering building. Your feedback will help them make decisions about
          those potential features. """

# Construct questions for the task
q1 = QuestionMultipleChoice(
    question_name = "use_often",
    question_text = context + """Consider the following new feature: {{ content }}
    How often do you think you would use it?""",
    question_options = ["Never", "Occasionally", "Frequently", "All the time"]
)

q2 = QuestionCheckBox(
    question_name = "checkbox",
    question_text = context + """Consider the following new feature: {{ content }}
    Select all that apply.""",
    question_options = [
        "This feature would be useful to me.",
        "This feature would make me more productive.",
        "This feature will be important to me.",
        "The benefits of this feature are not clear to me.",
        "I would like to see some examples of how to use this feature." 
    ]
)

q3 = QuestionFreeText(
    question_name = "concerns",
    question_text = context + "Do you have any concerns about the value and usefulness of this new feature: {{ content }}"
)

q4 = QuestionLinearScale(
    question_name = "likely_to_use",
    question_text = context + """Consider the following new feature: {{ content }}
    On a scale from 1 to 5, how likely are you to use this new feature? 
    (1 = not at all likely, 5 = very likely)""",
    question_options = [1,2,3,4,5]
)

# Create some content for the agent to review
contents = [
    "An optional progress bar that shows how many of your questions have been answered while your survey is running.",
    "A method that lets you quickly check what version of the package you have installed.",
    "A method that lets you include questions and responses as context for new questions."
]

# Parameterize the questions with the content 
scenarios = [Scenario({"content":c}) for c in contents]

# Create a survey with the questions
survey = Survey(questions = [q1, q2, q3, q4])

# Run the survey and store the results, and show a progress bar
results = survey.by(scenarios).by(agent).run(progress_bar=True)

Output()

In [4]:
# Show all columns of the Results object
results.columns

['agent.age',
 'agent.agent_name',
 'agent.company',
 'agent.hobbies',
 'agent.industry',
 'agent.location',
 'agent.occupation',
 'agent.persona',
 'answer.checkbox',
 'answer.checkbox_comment',
 'answer.concerns',
 'answer.likely_to_use',
 'answer.likely_to_use_comment',
 'answer.use_often',
 'answer.use_often_comment',
 'iteration.iteration',
 'model.frequency_penalty',
 'model.logprobs',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.temperature',
 'model.top_logprobs',
 'model.top_p',
 'prompt.checkbox_system_prompt',
 'prompt.checkbox_user_prompt',
 'prompt.concerns_system_prompt',
 'prompt.concerns_user_prompt',
 'prompt.likely_to_use_system_prompt',
 'prompt.likely_to_use_user_prompt',
 'prompt.use_often_system_prompt',
 'prompt.use_often_user_prompt',
 'raw_model_response.checkbox_raw_model_response',
 'raw_model_response.concerns_raw_model_response',
 'raw_model_response.likely_to_use_raw_model_response',
 'raw_model_response.use_often_raw_model_respo

In [5]:
# Just print the responses
results.select("scenario.*", "answer.use_often", "answer.checkbox", "answer.concerns", "answer.likely_to_use").print()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ scenario                 ┃ answer       ┃ answer                    ┃ answer                   ┃ answer         ┃
┃ .content                 ┃ .use_often   ┃ .checkbox                 ┃ .concerns                ┃ .likely_to_use ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ A method that lets you   │ Frequently   │ ['This feature would be   │ I believe adding a       │ 5              │
│ quickly check what       │              │ useful to me.', 'This     │ feature that allows      │                │
│ version of the package   │              │ feature would make me     │ users to quickly check   │                │
│ you have installed.      │              │ more productive.', 'This  │ the version of the       │                │
│                          │              │ feature will be important │ package they have        │                │
│                          │              │ to me.', 'I would like to │ installed is quite       │                │
│                          │              │ see some examples of how  │ practical and valuable.  │                │
│                          │              │ to use this feature.']    │ In the realm of software │                │
│                          │              │                           │ development and          │                │
│                          │              │                           │ research, especially     │                │
│                          │              │                           │ when dealing with open   │                │
│                          │              │                           │ source packages, it's    │                │
│                          │              │                           │ crucial to ensure that   │                │
│                          │              │                           │ everyone is using the    │                │
│                          │              │                           │ correct version. This is │                │
│                          │              │                           │ because different        │                │
│                          │              │                           │ versions can have        │                │
│                          │              │                           │ varying functionalities, │                │
│                          │              │                           │ bug fixes, and           │                │
│                          │              │                           │ performance              │                │
│                          │              │                           │ improvements. Having an  │                │
│                          │              │                           │ easy way to verify the   │                │
│                          │              │                           │ installed version helps  │                │
│                          │              │                           │ maintain consistency     │                │
│                          │              │                           │ across different         │                │
│                          │              │                           │ environments and can aid │                │
│                          │              │                           │ in troubleshooting       │                │
│                          │              │                           │ issues when they arise.  │                │
│                          │              │                           │ It also simplifies the   │                │
│                          │              │                           │ process of ensuring      │                │
│                          │              │             